# SQL-GRECO 

In [1]:
# Importing Libraries 
import pandas as pd
import sqlite3 as sql

# Read csv files using Pandas
campaign_desc= pd.read_csv(f"./campaign_desc.csv")
campaign_table = pd.read_csv(f"./campaign_table.csv")
coupon = pd.read_csv(f"./coupon.csv")
coupon_redempt = pd.read_csv(f"./coupon_redempt.csv")
hh_demographic = pd.read_csv(f"./hh_demographic.csv")
product = pd.read_csv(f"./product.csv")
transactions = pd.read_csv(f"./transactions.csv")

# Creating a SQLite Database Connection:
database = './Database.db'
connection_obj = sql.connect(database)
 
# Cursor Object
cursor_obj = connection_obj.cursor()

# Converting the DataFrames to SQL DB
campaign_desc.to_sql('campaign_desc', connection_obj, if_exists='replace', index = False)
campaign_table.to_sql('campaign_table', connection_obj, if_exists='replace', index = False)
coupon.to_sql('coupon', connection_obj, if_exists='replace', index = False)
coupon_redempt.to_sql('coupon_redempt', connection_obj, if_exists='replace', index = False)
hh_demographic.to_sql('hh_demographic', connection_obj, if_exists='replace', index = False)
product.to_sql('product', connection_obj, if_exists='replace', index = False)
transactions.to_sql('transactions', connection_obj, if_exists='replace', index = False)

312357

## Data dictionary ##

### Transaction ###
- `household_key`: The key indicating the household (customer) who bought the product
- `BASKET_ID`: The basket (bill) this item was sold as a part of
- `DAY`: Day (in absolute numbers)
- `PRODUCT_ID`: Product code
- `QUANTITY`: No. of units sold
- `SALES_VALUE`: Value of the sale in $
- `STORE_ID`: Store code
- `RETAIL_DISC`: Discount applied on the item (not as a part of a coupon/campaign)
- `TRANS_TIME`: Time of the sale (in HHMM format)
- `WEEK_NO`: Week number
- `COUPON_DISC`: Discount applied on the item as part of a coupon

### Product ###
- `PRODUCT_ID`: Product code
- `MANUFACTURER`: Manufacturer code
- `DEPARTMENT`: Department of the product
- `BRAND`: Brand type of the product – National or Private label
- `COMMODITY_DESC`: Product commodity type description
- `SUB_COMMODITY_DESC`: Product sub-commodity type description
- `CURR_SIZE_OF_PRODUCT`: Size of the product – In various units

### Coupon ###
- `COUPON_UPC`: Coupon Code
- `PRODUCT_ID`: Product that this code was applicable to
- `CAMPAIGN`: Campaign in which this coupon was offered

### Coupon_redempt ###
- `household_key`: The household (customer) who redeemed this coupon
- `DAY`: Day on which the coupon was redeemed
- `COUPON_UPC`: Coupon Code
- `CAMPAIGN`: Campaign in which this coupon was offered

### hh_demographic ###
- `AGE_DESC`: Age group of the household
- `INCOME_DESC`: Income range of the household
- `HOMEOWNER_DESC`: Housing status of the household – Renter/Homeowner etc.
- `HH_COMP_DESC`: Composition of the people in the household
- `HOUSEHOLD_SIZE_DESC`: Number of people in the household
- `KID_CATEGORY_DESC`: Number of kids in the household if any
- `household_key`: Household (customer) code

### Campaign_table ###
- `DESCRIPTION`: Type of the campaign
- `household_key`: Household(s) this campaign was applicable to
- `CAMPAIGN`: Campaign code

### Campaign_desc ###
- `DESCRIPTION`: Type of the campaign
- `CAMPAIGN`: Campaign code
- `START_DAY`: Start date of the campaign
- `END_DAY`: End date of the campaign


### Questions 

1.	Which are the top 3 manufacturers for the retailer in terms of number of products produced? * distinct 
2.	Total number of days that each type of campaign (Type A, Type B & Type C) ran. Which campaign 
    (Not desc, the campaign e.g., 8,26 etc.) ran for most days?
3.	How many coupons are mapped to each Campaign type? (e.g., Type A - 20 etc.)
4.	How much has the company spent on retail discounts and campaign-based discounts? Which is higher?
5.	What is the distribution of low, medium & high-income houses targeted by each type of campaign? 
    (Default - Low < 40K, Medium 40K to 80K & High 80K<) {you could change the default threshold, 
    just provide the reasoning for new threshold in the comments. Extra credits will be provided for improved thresholds}
6.	What is the distribution of purchase value for Single, Adults with no kids, Adults with kids’ households? 
7.	What are the total sales generated by products have no defined size of the product?
8.	Which is the busiest hour of sales across all stores? Justify your approach in comments.
9.	Assuming Day = 0 to be Jan 1, 2020, how much sales was made in December 2020?
10.	Find the number of households who have made purchases in any number of consecutive weeks
11.	Does the increase in number of kids in a household correspond to a higher average purchase value 
    of each purchase by a household? (Calculate for each purchase and not each item)
12.	Were there any number of coupons not redeemed by customers? If so, how many?
13.	What were the two items were most frequently purchased together? If there’s more than one, 
    display top 5 in order of product ID.
14.	Assuming cost to run all three types of campaign is same, which campaign turned out to be 
    most profitable for the retailer? Justify your approach in the comments
15. What is the average % increase/decrease in sales value (of each purchase by a household) in all purchases 
    following a purchase with a discount applied from a coupon? ** ( Omitted Question )

In [8]:
# 1. Which are the top 3 manufacturers for the retailer in terms of number of products produced? 
import pyodbc
import pandas as pd
# select manufacturer, rank() over (partiction by manufacturer order by count(*) desc) as product_count
# from product
# group by manufacturer
# where product_count <= 3

sql_code = """
select manufacturer, count(*) as product_count
from product
group by manufacturer
order by product_count desc
limit 3;
"""

cursor_obj.execute(sql_code) # Executing the code 
connection_obj.commit() #  Commit changes (if any)
results = cursor_obj.fetchall() # Fetch all the results

df=pd.DataFrame(results, columns=['manufacturer', 'product_count']) # Convert results to DataFrame
df

,manufacturer,product_count
0,69,12676
1,2,1411
2,5423,1376


In [1]:
pip install pyodbc

   ---------------------------------------- 0.0/69.5 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/69.5 kB ? eta -:--:--
   ----------------- ---------------------- 30.7/69.5 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------- 69.5/69.5 kB 955.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
# 2. Which campaign (Not desc, the campaign e.g., 8,26 etc.) ran for most days?

sql_code = """
WITH CTE AS (
    SELECT 
        description,
        JULIANDAY(END_DAY) - JULIANDAY(START_DAY) AS days
    FROM campaign_desc
)

SELECT 
    description, 
    SUM(days) AS days_count
FROM CTE
GROUP BY description
HAVING SUM(days) = (
    SELECT MAX(total_days)
    FROM (
        SELECT 
            description, 
            SUM(days) AS total_days
        FROM CTE
        GROUP BY description
    )
);

"""
##

cursor_obj.execute(sql_code) # Executing the code 
connection_obj.commit() #  Commit changes (if any)
results = cursor_obj.fetchall() # Fetch all the results

df=pd.DataFrame(results, columns=['description', 'days_count']) # Convert results to DataFrame
df

,description,days_count
0,TypeB,715.0


In [13]:
# 2.1 Total number of days that each type of campaign (Type A, Type B & Type C) ran.

sql_code = """
SELECT 
    description, 
    SUM(JULIANDAY(END_DAY) - JULIANDAY(START_DAY)) AS total_days
FROM 
    campaign_desc
GROUP BY 
    description;

"""

cursor_obj.execute(sql_code) # Executing the code 
connection_obj.commit() #  Commit changes (if any)
results = cursor_obj.fetchall() # Fetch all the results

df=pd.DataFrame(results, columns=['description', 'total_days']) # Convert results to DataFrame
df

,description,total_days
0,TypeA,236.0
1,TypeB,715.0
2,TypeC,447.0


In [24]:
# 3. How many coupons are mapped to each Campaign type? (e.g., Type A - 20 etc.)

sql_code = """
select a.DESCRIPTION as campaign_type
, count(*) as coupon_count
from 
    campaign_table a
join 
    coupon b
    on a.CAMPAIGN = b.CAMPAIGN
group by 
    a.DESCRIPTION
"""

cursor_obj.execute(sql_code) # Executing the code 
connection_obj.commit() #  Commit changes (if any)
results = cursor_obj.fetchall() # Fetch all the results

df=pd.DataFrame(results, columns=['campaign_type', 'coupon_count']) # Convert results to DataFrame
df

,campaign_type,coupon_count
0,TypeA,1482266
1,TypeB,160447
2,TypeC,10050


In [34]:
# 4 How much has the company spent on retail discounts and campaign-based discounts? Which is higher?

sql_code = """
select 
    a.product_id,
    b.BRAND,
    sum (a.RETAIL_DISC) as retail_discounts,
    sum(a.COUPON_DISC) as campaign_discounts
from 
    transactions a
    join
    product b
    on a.product_id = b.product_id
where 
    RETAIL_DISC is not null or COUPON_DISC is not null
group by a.product_id, b.BRAND
order by a.product_id DESC
limit 1

"""

cursor_obj.execute(sql_code) # Executing the code 
connection_obj.commit() #  Commit changes (if any)
results = cursor_obj.fetchall() # Fetch all the results

df=pd.DataFrame(results, columns=['product_id','brand', 'retail_discounts', 'campaign_discounts']) # Convert results to DataFrame
df

,product_id,brand,retail_discounts,campaign_discounts
0,14077546,Private,0.0,0.0


In [ ]:
# 5 What is the distribution of low, medium & high-income houses targeted by each type of campaign? 
# (Default - Low < 40K, Medium 40K to 80K & High 80K<) 
# {you could change the default threshold, just provide the reasoning for new threshold in the comments. 
# Extra credits will be provided for improved thresholds}

sql_code = """
select b.DESCRIPTION as campaign_type,
    case when INCOME_DESC < 40000 then 'Low'
         when INCOME_DESC >= 40000 and INCOME_DESC < 80000 then 'Medium'
            when INCOME_DESC >= 80000 then 'High'
            else 'Unknown' end as income_group ,
    count(*) as household_count
from 
    hh_demographic a
join 
    campaign_table b
using (household_key)
group by campaign_type
, income_group


        
"""

cursor_obj.execute(sql_code) # Executing the code 
connection_obj.commit() #  Commit changes (if any)
results = cursor_obj.fetchall() # Fetch all the results

df=pd.DataFrame(results, columns=['campaign_type', 'income_group', 'household_count']) # Convert results to DataFrame
df

,campaign_type,income_group,household_count
0,TypeA,High,172
1,TypeA,Low,1217
2,TypeA,Medium,790
3,TypeB,High,114
4,TypeB,Low,934
5,TypeB,Medium,624
6,TypeC,High,35
7,TypeC,Low,171
8,TypeC,Medium,156


In [66]:
# 6. What is the distribution of purchase value for Single, Adults with no kids, Adults with kids’ households?

sql_code = """
select 
  
    case 
        when cast(replace(HOUSEHOLD_SIZE_DESC, '+', '') as integer) = 1 
            and KID_CATEGORY_DESC = 'None/Unknown' then 'Single'
        when cast(replace(HOUSEHOLD_SIZE_DESC, '+', '') as integer) > 1 
            and KID_CATEGORY_DESC = 'None/Unknown' then 'Adults with no kids'
    when cast(replace(KID_CATEGORY_DESC, '+', '') as integer) >= 1 
            or KID_CATEGORY_DESC in ('1', '2', '3+') then 'Adults with kids'
        else 'Unknown'
    end as household_type,
    sum(b.SALES_VALUE) as total_sales
from 
    hh_demographic a
join 
    transactions b
using 
    (household_key)
group by 
    1;
    
"""

cursor_obj.execute(sql_code) # Executing the code 
connection_obj.commit() #  Commit changes (if any)
results = cursor_obj.fetchall() # Fetch all the results

df=pd.DataFrame(results, columns=[ 'household_type','sum']) # Convert results to DataFrame
df

,household_type,sum
0,Adults with kids,188371.68
1,Adults with no kids,197153.86
2,Single,152302.69


In [ ]:
# ***7. What are the total sales generated by products have no defined size of the product? 

sql_code = """
select a.PRODUCT_ID, sum(a.SALES_VALUE) as total
from 
    transactions a
join    
    product b
using
    (product_id)
where 
    b.CURR_SIZE_OF_PRODUCT is null or b.CURR_SIZE_OF_PRODUCT = ''

"""

cursor_obj.execute(sql_code) # Executing the code 
connection_obj.commit() #  Commit changes (if any)
results = cursor_obj.fetchall() # Fetch all the results

df=pd.DataFrame(results, columns=['product_id', 'total_sales']) # Convert results to DataFrame
df

,product_id,total_sales
0,None,None


In [45]:
# 8. Which is the busiest hour of sales across all stores? Justify your approach in comments.

sql_code = """
--take the transactions table and extract the hour from and converting the transaction time
--  Get transaction counts per store and hour

with CTE as(   
    Select 
      store_id,
      TRANS_TIME / 100 AS hour,
      COUNT(*) AS total_transactions
    from transactions
    group by store_id, hour
    )

select 
  t1.store_id,
  t1.hour as max_hour,
  t1.total_transactions
from CTE t1                              
where t1.total_transactions = (                                          
  select MAX(t2.total_transactions)     --  Get the maximum total_transactions for each store
  from CTE t2
  where t1.store_id = t2.store_id       --  Compare with the same store
);

"""

cursor_obj.execute(sql_code) # Executing the code 
connection_obj.commit() #  Commit changes (if any)
results = cursor_obj.fetchall() # Fetch all the results

df=pd.DataFrame(results, columns=['store_id','max_hour','total_transactions']) # Convert results to DataFrame
df

,store_id,max_hour,total_transactions
0,12,21,4
1,20,16,2
2,27,15,40
3,42,8,7
4,55,17,30
...,...,...,...
244,32004,17,620
245,32064,18,17
246,33863,18,26
247,33904,17,46


In [11]:
# 9. Assuming Day = 0 to be Jan 1, 2020, how much sales was made in December 2020?

sql_code = """
select 
    sum(SALES_VALUE) as total_sales
from transactions 
where 
    DAY >= 334 and DAY <= 366; 
"""

cursor_obj.execute(sql_code) # Executing the code 
connection_obj.commit() #  Commit changes (if any)
results = cursor_obj.fetchall() # Fetch all the results

df=pd.DataFrame(results ,columns=['total_sales'])
df

,total_sales
0,401545.93


In [28]:
# 10. Find the number of households who have made purchases in any number of consecutive weeks

sql_code = """
select 
   count( distinct a.household_key) as count_households
  
from transactions a
join
transactions b
on a.household_key = b.household_key and 
    a.WEEK_NO = b.WEEK_NO - 1


"""

cursor_obj.execute(sql_code) # Executing the code 
connection_obj.commit() #  Commit changes (if any)
results = cursor_obj.fetchall() # Fetch all the results

df=pd.DataFrame(results, columns=['count_households']) # Convert results to DataFrame
df

,count_households
0,1793


In [1]:
# 11. Does the increase in number of kids in a household correspond to a higher 
# average purchase value of each purchase by a household? (Calculate for each purchase and not each item) 

sql_code = """
select
    a.KID_CATEGORY_DESC,
    avg(b.SALES_VALUE) as avg_purchase_value
from 
    hh_demographic a
join
    transactions b
using (household_key)
group by a.KID_CATEGORY_DESC
order by avg_purchase_value desc
"""

cursor_obj.execute(sql_code) # Executing the code 
connection_obj.commit() #  Commit changes (if any)
results = cursor_obj.fetchall() # Fetch all the results

df=pd.DataFrame(results)
df

NameError: name 'cursor_obj' is not defined

In [ ]:
# 12. Were there any number of coupons not redeemed by customers? If so, how many?

sql_code = """
select 
    count(*) as unredeemed_coupons  
from coupon
where coupon_id not in (select coupon_id from coupon_redempt)
"""

cursor_obj.execute(sql_code) # Executing the code 
connection_obj.commit() #  Commit changes (if any)
results = cursor_obj.fetchall() # Fetch all the results

df=pd.DataFrame(results)
df

""


In [ ]:
# 13. What were the two items were most frequently purchased together? 
# If there’s more than one, display top 5 in order of product ID.

sql_code = """
select 
    a.product_id as product1, 
    b.product_id as product2, 
    count(*) as purchase_count
from
    transactions a
join
    transactions b
on a.household_key = b.household_key
where a.product_id < b.product_id
group by a.product_id, b.product_id
order by purchase_count desc, product1, product2
"""

cursor_obj.execute(sql_code) # Executing the code 
connection_obj.commit() #  Commit changes (if any)
results = cursor_obj.fetchall() # Fetch all the results

df=pd.DataFrame(results)
df

""


In [ ]:
# 14. Assuming cost to run all three types of campaign is same, 
# which campaign turned out to be most profitable for the retailer? Justify your approach in the comments

sql_code = """
select 
    a.DESCRIPTION as campaign_type,
    sum(b.SALES_VALUE) as total_sales,
    sum(b.COUPON_DISC) as total_coupon_discount,
    sum(b.RETAIL_DISC) as total_retail_discount,
    (sum(b.SALES_VALUE) - sum(b.COUPON_DISC) - sum(b.RETAIL_DISC)) as profit
from
    campaign_table a
join
    transactions b
using (household_key)
group by a.DESCRIPTION
order by profit desc
"""

cursor_obj.execute(sql_code) # Executing the code 
connection_obj.commit() #  Commit changes (if any)
results = cursor_obj.fetchall() # Fetch all the results

df=pd.DataFrame(results)
df

""


In [ ]:
# 15. What is the average % increase/decrease in sales value (of each purchase by a household) in all purchases 
#     following a purchase with a discount applied from a coupon?

sql_code = """
select 
    a.household_key,
    avg((b.SALES_VALUE - a.SALES_VALUE) / a.SALES_VALUE * 100) as avg_percentage_change
from
    transactions a
join

    transactions b
on a.household_key = b.household_key
where a.COUPON_DISC > 0
group by a.household_key
order by avg_percentage_change desc

"""

cursor_obj.execute(sql_code) # Executing the code 
connection_obj.commit() #  Commit changes (if any)
results = cursor_obj.fetchall() # Fetch all the results

df=pd.DataFrame(results)
df

""
